In [57]:
# Om te importen moet je een terminal openen (vs code) en de volende code invullen: py -m pip install numpy
import pandas as pd
import numpy as np
import matplotlib as plt

pd.set_option("display.max.columns", None) #Make sure all columns are showed

df_train = pd.read_csv("H:/Mijn documenten/Python/kaggle/titanic_competition/train.csv")
df_test = pd.read_csv("H:/Mijn documenten/Python/kaggle/titanic_competition/test.csv")
df_train.columns = df_train.columns.str.lower()
df_test.columns = df_test.columns.str.lower()

print(df_train.info())
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   passengerid  891 non-null    int64  
 1   survived     891 non-null    int64  
 2   pclass       891 non-null    int64  
 3   name         891 non-null    object 
 4   sex          891 non-null    object 
 5   age          714 non-null    float64
 6   sibsp        891 non-null    int64  
 7   parch        891 non-null    int64  
 8   ticket       891 non-null    object 
 9   fare         891 non-null    float64
 10  cabin        204 non-null    object 
 11  embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   passengerid  418 non-null    int64  
 1   pcl

## Restructure data

In [69]:
# Categoriseren
cat_list =  ["sex", "pclass", "embarked"]
for x in cat_list:
  df_train[x] = df_train[x].astype("category")


<class 'pandas.core.series.Series'>
RangeIndex: 891 entries, 0 to 890
Series name: age
Non-Null Count  Dtype  
--------------  -----  
714 non-null    float64
dtypes: float64(1)
memory usage: 7.1 KB
male      577
female    314
Name: sex_cat, dtype: int64
category


In [73]:
# Groeperen:
## Age -> agegroups
df_train.age.describe()

bins = [0, 24.999, 74.999, 100]
group_names = [1,2,3]
df_train['age_group'] = pd.cut(df_train['age'], bins, labels=group_names)



In [ ]:
# Check kinderen onder 18 en relatie tot ouders

ft = df_train.sex_cat.value_counts()
pd.crosstab(index=df_train['survived'], columns=df_train['sex_cat'],
 dropna=False, normalize=False, margins = True, margins_name= "Total")

dt = df_train.sex_cat.dtype